<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e9a57e41e420df02795bf62b4af36e3f215e421c872d41c63c71ae1b08ccfafb
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-29 10:05:09
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.37 C
-------------------
Today PnL: 60.25 K (0.44%)
Current PnL: -22.70 L (-15.27%)
CY Booked + Current PnL: -11.26 L (-7.58%)
-------------------
Total profit:  95.58 K
Total loss:  -23.66 L
-------------------
Total Booked + Current PnL: 15.72 L (12.91%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.36%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 88.57 L (64.7%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.1%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,94.45,39.0,M-SC,2.46,83801.0,-16976.0,17070.0,0.75,-16.85,20.37,0.09,245.0,-0.99,0.61,10.10,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.17,46.0,H-MC,3.61,188430.0,23760.0,19578.0,-0.31,14.43,10.39,26.31,89.0,1.21,1.38,23.56,X40N,NTT,AC
50,NATIONALUM,244.55,-45.32,52.0,H-MC,11.87,108284.0,7780.0,21332.0,1.86,7.74,19.70,28.96,79.0,0.36,0.79,44.34,MH,ATH,METALS
42,ITC,452.00,-38.34,47.0,H-LC,1.91,198583.0,-1555.0,22440.0,0.25,-0.78,11.30,10.44,4.0,-0.07,1.45,4.74,X40,NTT,FMCG
57,RELIANCE,1533.00,-13.66,47.0,H-LC,3.94,215998.0,5632.0,23155.0,0.51,2.68,10.72,13.68,37.0,0.24,1.58,19.64,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-177.27,70.0,M-SC,6.60,89466.0,-12744.0,106134.0,8.19,-12.47,118.63,91.37,208.0,-0.12,0.65,45.23,XY25,NTT,FINANCE
32,HINDZINC,730.22,23.40,57.0,M-LC,2.93,200883.0,-4193.0,116753.0,2.83,-2.04,58.12,54.89,52.0,-0.04,1.47,21.56,X5K,ATH,METALS
10,BANDHANBNK,400.00,-14.68,42.0,H-SC,4.13,213301.0,-65258.0,322703.0,2.68,-23.43,151.29,92.42,151.0,-0.20,1.56,23.27,XY24,NTT,BANKS
64,SHALBY,327.00,1188.68,65.0,M-SC,14.01,175204.0,-5939.0,51405.0,2.30,-3.28,29.34,25.10,235.0,-0.12,1.28,40.41,XY24,NTT,HEALTHCARE
6,ASIANTILES,137.00,6622.22,46.0,L-SC,5.30,73782.0,-20028.0,96647.0,2.17,-21.35,130.99,81.67,269.0,-0.21,0.54,42.88,XR,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ANGELONE,3033.00,-1.02,29.0,H-SC,11.13,175185.0,-15821.0,73525.0,-2.67,-8.28,41.97,30.21,157.0,-0.22,1.28,10.87,X40N,NTT,FINANCE
38,INDIGOPNTS,1408.00,102.01,30.0,M-SC,1.52,132308.0,-42251.0,42286.0,-2.16,-24.20,31.96,0.02,221.0,-1.00,0.97,15.09,OX40N,NTT,PAINTS
13,BERGEPAINT,680.00,-23.00,34.0,H-MC,4.23,207036.0,-20303.0,68363.0,-1.97,-8.93,33.02,21.14,106.0,-0.30,1.51,16.70,XY24,NTT,PAINTS
71,TANLA,1963.11,-21.97,49.0,H-SC,17.41,209013.0,-61748.0,391690.0,-1.48,-22.81,187.40,121.86,133.0,-0.16,1.53,63.51,AR,ATH,IT
15,BSOFT,836.99,-19.05,33.0,H-SC,13.77,90958.0,-49695.0,122475.0,-1.23,-35.33,134.65,51.74,131.0,-0.41,0.66,3.73,XR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.0,-45.46,29.0,M-SC,1.41,92280.0,782.0,77967.0,-0.18,0.85,84.49,86.07,223.0,0.01,0.67,34.01,XR,NTT,DURABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,92.53,51.0,H-SC,10.32,121824.0,-20655.0,40178.0,2.13,-14.50,32.98,13.70,163.0,-0.51,0.89,43.79,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,94.45,39.0,M-SC,2.46,83801.0,-16976.0,17070.0,0.75,-16.85,20.37,0.09,245.0,-0.99,0.61,10.10,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,50.0,H-SC,1.53,220950.0,-48717.0,85044.0,-0.44,-18.07,38.49,13.47,138.0,-0.57,1.61,11.75,XY24,NTT,PAINTS
18,CERA,9475.0,-22.43,37.0,H-SC,1.69,141197.0,-34706.0,76726.0,-0.66,-19.73,54.34,23.89,149.0,-0.45,1.03,21.94,OX40N,NTT,CERAMICS
66,SIS,528.0,2082.90,40.0,H-SC,4.02,88090.0,-22942.0,46547.0,0.07,-20.66,52.84,21.26,156.0,-0.49,0.64,19.02,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,56.0,M-MC,10.74,229336.0,4374.0,163287.0,1.65,1.94,71.20,74.53,192.0,0.03,1.68,34.82,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,9.17,46.0,H-MC,3.61,188430.0,23760.0,19578.0,-0.31,14.43,10.39,26.31,89.0,1.21,1.38,23.56,X40N,NTT,AC
84,VOLTAS,1530.00,-3.32,42.0,H-MC,2.84,202875.0,11133.0,26617.0,0.92,5.81,13.12,19.69,99.0,0.42,1.48,13.00,XY25,NTT,AC
17,CAMS,5211.76,-5.83,44.0,H-SC,1.79,106996.0,4992.0,38936.0,0.56,4.89,36.39,43.06,122.0,0.13,0.78,22.85,X40N,ATH,MISC
79,UNIONBANK,163.00,-14.73,53.0,M-LC,7.03,154803.0,13963.0,30527.0,1.54,9.91,19.72,31.59,66.0,0.46,1.13,39.03,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-45.32,52.0,H-MC,11.87,108284.0,7780.0,21332.0,1.86,7.74,19.70,28.96,79.0,0.36,0.79,44.34,MH,ATH,METALS
11,BANKINDIA,190.00,-29.19,56.0,H-MC,8.45,182776.0,2968.0,109446.0,2.16,1.65,59.88,62.52,88.0,0.03,1.34,36.30,XR,NTT,BANKS
37,INDIAMART,4850.92,-55.01,26.0,H-SC,9.99,125584.0,2248.0,131512.0,0.11,1.82,104.72,108.45,119.0,0.02,0.92,24.58,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-45.46,29.0,M-SC,1.41,92280.0,782.0,77967.0,-0.18,0.85,84.49,86.07,223.0,0.01,0.67,34.01,XR,NTT,DURABLES
86,WIPRO,420.00,-15.61,38.0,M-LC,5.46,148180.0,-2765.0,112217.0,1.38,-1.83,75.73,72.51,53.0,-0.02,1.08,4.08,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.03,30.0,H-LC,6.53,207002.0,-44766.0,97974.0,0.41,-17.78,47.33,21.13,27.0,-0.46,1.51,11.35,X40,ATH,PAINTS
83,VBL,671.64,-20.80,31.0,H-LC,8.52,285090.0,-30752.0,143429.0,0.47,-9.74,50.31,35.67,5.0,-0.21,2.08,2.77,X40N,ATH,FMCG
74,TCS,4476.75,-31.26,32.0,H-LC,12.15,274339.0,-71617.0,146470.0,0.67,-20.70,53.39,21.64,1.0,-0.49,2.00,0.67,X40,ATH,IT
44,JIOFIN,387.00,-5.53,32.0,H-LC,12.81,256624.0,-10741.0,76192.0,0.86,-4.02,29.69,24.48,48.0,-0.14,1.88,48.75,XY24,BTT,FINANCE
30,HCLTECH,1943.91,-4.09,34.0,H-LC,10.00,219172.0,-22752.0,86025.0,0.05,-9.40,39.25,26.15,8.0,-0.26,1.60,3.57,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-7.51,41.0,H-LC,1.58,229590.0,-16561.0,95762.0,0.53,-6.73,41.71,32.18,16.0,-0.17,1.68,26.89,X200,ATH,IT
42,ITC,452.0,-38.34,47.0,H-LC,1.91,198583.0,-1555.0,22440.0,0.25,-0.78,11.30,10.44,4.0,-0.07,1.45,4.74,X40,NTT,FMCG
65,SIEMENS,4671.5,-2.40,45.0,H-LC,2.39,156795.0,-29300.0,76783.0,1.28,-15.74,48.97,25.51,15.0,-0.38,1.15,15.55,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-13.66,47.0,H-LC,3.94,215998.0,5632.0,23155.0,0.51,2.68,10.72,13.68,37.0,0.24,1.58,19.64,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-21.84,43.0,H-LC,4.33,208240.0,3740.0,37671.0,1.37,1.83,18.09,20.25,10.0,0.10,1.52,10.72,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,32.0,H-LC,12.15,274339.0,-71617.0,146470.0,0.67,-20.70,53.39,21.64,1.0,-0.49,2.00,0.67,X40,ATH,IT
80,UNITDSPR,1644.00,-14.09,45.0,H-LC,7.20,225045.0,-9099.0,57724.0,0.85,-3.89,25.65,20.77,86.0,-0.16,1.64,2.76,X40N,NTT,BREWERIES
83,VBL,671.64,-20.80,31.0,H-LC,8.52,285090.0,-30752.0,143429.0,0.47,-9.74,50.31,35.67,5.0,-0.21,2.08,2.77,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.09,34.0,H-LC,10.00,219172.0,-22752.0,86025.0,0.05,-9.40,39.25,26.15,8.0,-0.26,1.60,3.57,X40,ATH,IT
42,ITC,452.00,-38.34,47.0,H-LC,1.91,198583.0,-1555.0,22440.0,0.25,-0.78,11.30,10.44,4.0,-0.07,1.45,4.74,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.40,45.0,H-LC,2.39,156795.0,-29300.0,76783.0,1.28,-15.74,48.97,25.51,15.0,-0.38,1.15,15.55,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.34,47.0,H-LC,1.91,198583.0,-1555.0,22440.0,0.25,-0.78,11.30,10.44,4.0,-0.07,1.45,4.74,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.03,30.0,H-LC,6.53,207002.0,-44766.0,97974.0,0.41,-17.78,47.33,21.13,27.0,-0.46,1.51,11.35,X40,ATH,PAINTS
19,CIPLA,1795.00,-21.84,43.0,H-LC,4.33,208240.0,3740.0,37671.0,1.37,1.83,18.09,20.25,10.0,0.10,1.52,10.72,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-13.66,47.0,H-LC,3.94,215998.0,5632.0,23155.0,0.51,2.68,10.72,13.68,37.0,0.24,1.58,19.64,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,32.0,H-LC,12.15,274339.0,-71617.0,146470.0,0.67,-20.70,53.39,21.64,1.0,-0.49,2.00,0.67,X40,ATH,IT
40,INFY,2275.00,-20.32,42.0,H-LC,8.18,311023.0,-1865.0,173551.0,0.78,-0.60,55.80,54.87,3.0,-0.01,2.27,6.01,X40,BTT,IT
42,ITC,452.00,-38.34,47.0,H-LC,1.91,198583.0,-1555.0,22440.0,0.25,-0.78,11.30,10.44,4.0,-0.07,1.45,4.74,X40,NTT,FMCG
83,VBL,671.64,-20.80,31.0,H-LC,8.52,285090.0,-30752.0,143429.0,0.47,-9.74,50.31,35.67,5.0,-0.21,2.08,2.77,X40N,ATH,FMCG
1,ABB,7934.00,-39.96,48.0,H-LC,7.68,251078.0,-10541.0,129757.0,1.11,-4.03,51.68,45.57,7.0,-0.08,1.83,6.84,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,44.0,L-SC,40.32,79296.0,-34253.0,74300.0,2.15,-30.17,93.70,35.27,268.0,-0.46,0.58,94.08,XR,NTT,HOTELS
61,SAMMAANCAP,326.00,-177.27,70.0,M-SC,6.60,89466.0,-12744.0,106134.0,8.19,-12.47,118.63,91.37,208.0,-0.12,0.65,45.23,XY25,NTT,FINANCE
49,MASFIN,398.61,-20.51,37.0,H-SC,12.32,90540.0,-7440.0,29045.0,-0.28,-7.59,32.08,22.05,152.0,-0.26,0.66,31.61,XR,ATH,FINANCE
50,NATIONALUM,244.55,-45.32,52.0,H-MC,11.87,108284.0,7780.0,21332.0,1.86,7.74,19.70,28.96,79.0,0.36,0.79,44.34,MH,ATH,METALS
78,UJJIVANSFB,60.00,92.53,51.0,H-SC,10.32,121824.0,-20655.0,40178.0,2.13,-14.50,32.98,13.70,163.0,-0.51,0.89,43.79,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1188.68,65.0,M-SC,14.01,175204.0,-5939.0,51405.0,2.30,-3.28,29.34,25.10,235.0,-0.12,1.28,40.41,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.0,-177.27,70.0,M-SC,6.60,89466.0,-12744.0,106134.0,8.19,-12.47,118.63,91.37,208.0,-0.12,0.65,45.23,XY25,NTT,FINANCE
81,VAIBHAVGBL,521.0,51.52,48.0,H-SC,5.75,131169.0,-51606.0,162676.0,0.04,-28.23,124.02,60.77,125.0,-0.32,0.96,17.48,XR,NTT,JEWELLERY
60,SAIL,228.0,45.15,56.0,M-MC,10.74,229336.0,4374.0,163287.0,1.65,1.94,71.20,74.53,192.0,0.03,1.68,34.82,XY24,BTT,STEEL
7,ATULAUTO,844.0,3548.65,48.0,M-SC,7.44,113823.0,-33545.0,85356.0,0.92,-22.76,74.99,35.16,236.0,-0.39,0.83,16.26,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.25
1,25,43.67
2,50,74.90


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.80
LC    32.53
MC    23.63
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.34
X40      14.76
X40N     11.40
XR       11.13
XY25     10.19
AR        8.30
OX40N     7.86
X200      1.68
MH        1.62
X5K       1.47
SR        1.21
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.28
H-LC    25.99
H-MC    20.67
M-SC    13.05
M-LC     5.48
M-MC     2.66
L-SC     1.47
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.04,-6.82,42.07
IT,12.78,-20.09,85.57
FINANCE,10.09,-18.69,70.47
MISC,6.97,-20.29,87.17
BANKS,6.21,-17.54,79.38
PAINTS,5.60,-20.34,38.27
ELECTRICAL,5.50,-9.43,47.82
INSURANCE,3.77,-7.10,47.14
AC,3.57,0.84,15.77


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3225548.0,22
XR,1386626.0,14
AR,1196712.0,9
X40,827770.0,10
X40N,643661.0,9
OX40N,570331.0,10
XY25,445273.0,7
SR,256011.0,2
X5K,116753.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3784745.0,29
M-SC,1458580.0,17
H-MC,1276312.0,15
H-LC,1261894.0,15
M-LC,400365.0,4
M-MC,314325.0,2
L-SC,259866.0,3
L-MC,61365.0,1
L-LC,39501.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1247056.0      6
M-SC       XY24       835891.0      7
H-SC       AR         820157.0      5
           XR         806819.0      7
H-MC       XY24       580689.0      4
H-LC       X40        526460.0      5
H-SC       X40N       318348.0      4
M-MC       XY24       314325.0      2
H-SC       OX40N      265180.0      4
           SR         256011.0      2
H-LC       X40N       238824.0      3
H-MC       X40        224304.0      4
H-LC       AR         206540.0      2
H-MC       XY25       181522.0      2
M-LC       XY24       171395.0      2
L-SC       XR         170947.0      2
M-SC       AR         170015.0      2
           OX40N      143702.0      4
           XR         125832.0      2
H-LC       XY25       118116.0      3
M-LC       X5K        116753.0      1
           XR         112217.0      1
H-MC       XR         109446.0      1
M-SC       XY25       106134.0      1
H-LC       X200        95762.0      1
L-SC       OX40N       88919.0      1
H-MC       X40N        86489.0      2
M-SC       X40         77006.0      1
H-LC       XY24        76192.0      1
H-MC       OX40N       72530.0      1
H-SC       MH          71174.0      1
L-MC       XR          61365.0      1
L-LC       XY25        39501.0      1
H-MC       MH          21332.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
